# Telecom Customer Data Preprocessing
## AI Customer Upsell Prediction System

This notebook handles:
- Data loading and cleaning
- Duplicate removal
- Missing value treatment
- Feature engineering (43 advanced features)
- Data quality assessment

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8')
sns.set_palette('viridis')

/home/reddy007/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load the telecom dataset
print("📊 Loading Telecom Customer Dataset...")
df_raw = pd.read_csv('../data/raw/telecom_data.csv')

print(f"Dataset Shape: {df_raw.shape}")
print(f"Columns: {list(df_raw.columns)}")
print("\n📋 First 5 rows:")
df_raw.head()

📊 Loading Telecom Customer Dataset...


Dataset Shape: (101174, 17)
Columns: ['Phone Number', 'Account Length', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn']

📋 First 5 rows:


,Phone Number,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
0,382-4657,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,371-7191,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,358-1921,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,375-9999,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,330-6626,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [3]:
# Data Quality Assessment
print("🔍 Data Quality Assessment")
print("="*50)

# Basic info
print(f"Total Records: {len(df_raw):,}")
print(f"Total Features: {df_raw.shape[1]}")
print(f"Memory Usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Data types
print("\n📊 Data Types:")
print(df_raw.dtypes.value_counts())

# Missing values
print("\n❌ Missing Values:")
missing_data = df_raw.isnull().sum()
missing_percent = (missing_data / len(df_raw)) * 100
missing_df = pd.DataFrame({
    'Missing Count': missing_data,
    'Missing Percentage': missing_percent
})
print(missing_df[missing_df['Missing Count'] > 0])

# Duplicates
duplicates = df_raw.duplicated().sum()
print(f"\n🔄 Duplicate Records: {duplicates:,} ({duplicates/len(df_raw)*100:.2f}%)")

🔍 Data Quality Assessment
Total Records: 101,174
Total Features: 17
Memory Usage: 17.95 MB

📊 Data Types:
float64    8
int64      7
object     1
bool       1
Name: count, dtype: int64

❌ Missing Values:
Empty DataFrame
Columns: [Missing Count, Missing Percentage]
Index: []

🔄 Duplicate Records: 40,729 (40.26%)


In [4]:
# Data Cleaning Process
print("🧹 Starting Data Cleaning Process...")
df_clean = df_raw.copy()

# Remove duplicates
print(f"Before duplicate removal: {len(df_clean):,} records")
df_clean = df_clean.drop_duplicates()
print(f"After duplicate removal: {len(df_clean):,} records")
print(f"Removed: {len(df_raw) - len(df_clean):,} duplicates")

# Handle missing values
numeric_columns = df_clean.select_dtypes(include=[np.number]).columns
for col in numeric_columns:
    if df_clean[col].isnull().sum() > 0:
        median_val = df_clean[col].median()
        df_clean[col].fillna(median_val, inplace=True)
        print(f"Filled {col} missing values with median: {median_val}")

# Convert Churn to binary
if 'Churn' in df_clean.columns:
    df_clean['Churn_Binary'] = df_clean['Churn'].map({
        'FALSE': 0, 'True': 1, False: 0, True: 1
    })
    df_clean['Churn_Binary'] = df_clean['Churn_Binary'].fillna(0).astype(int)
    print(f"Churn rate: {df_clean['Churn_Binary'].mean():.3f}")

print("\n✅ Data cleaning completed!")

🧹 Starting Data Cleaning Process...
Before duplicate removal: 101,174 records
After duplicate removal: 60,445 records
Removed: 40,729 duplicates
Churn rate: 0.110

✅ Data cleaning completed!


In [5]:
# Advanced Feature Engineering
print("🔧 Engineering Advanced Features...")
df_features = df_clean.copy() # df_features is now correctly based on df_clean (which is duplicate-free)

# --- CRUCIAL ADDITION: Re-confirm df_features is duplicate-free --- 
df_features = df_features.drop_duplicates()
print(f"Re-confirmed df_features are duplicate-free: {len(df_features):,} records")
# --- END ADDITION --- 

# Usage Aggregation Features
df_features['Total_Minutes'] = (df_features['Day Mins'] + df_features['Eve Mins'] + 
                               df_features['Night Mins'] + df_features['Intl Mins'])
df_features['Total_Calls'] = (df_features['Day Calls'] + df_features['Eve Calls'] + 
                             df_features['Night Calls'] + df_features['Intl Calls'])
df_features['Total_Charges'] = (df_features['Day Charge'] + df_features['Eve Charge'] + 
                               df_features['Night Charge'] + df_features['Intl Charge'])

# Usage Intensity Features
df_features['Avg_Call_Duration'] = df_features['Total_Minutes'] / (df_features['Total_Calls'] + 1e-6)
df_features['Day_Call_Duration'] = df_features['Day Mins'] / (df_features['Day Calls'] + 1e-6)
df_features['Eve_Call_Duration'] = df_features['Eve Mins'] / (df_features['Eve Calls'] + 1e-6)
df_features['Night_Call_Duration'] = df_features['Night Mins'] / (df_features['Night Calls'] + 1e-6)

# Usage Pattern Ratios
df_features['Day_Usage_Ratio'] = df_features['Day Mins'] / (df_features['Total_Minutes'] + 1e-6)
df_features['Eve_Usage_Ratio'] = df_features['Eve Mins'] / (df_features['Total_Minutes'] + 1e-6)
df_features['Night_Usage_Ratio'] = df_features['Night Mins'] / (df_features['Total_Minutes'] + 1e-6)
df_features['Intl_Usage_Ratio'] = df_features['Intl Mins'] / (df_features['Total_Minutes'] + 1e-6)

# Revenue-Based Features
df_features['Revenue_Per_Minute'] = df_features['Total_Charges'] / (df_features['Total_Minutes'] + 1e-6)
df_features['Day_Revenue_Rate'] = df_features['Day Charge'] / (df_features['Day Mins'] + 1e-6)
df_features['Eve_Revenue_Rate'] = df_features['Eve Charge'] / (df_features['Eve Mins'] + 1e-6)
df_features['Night_Revenue_Rate'] = df_features['Night Charge'] / (df_features['Night Mins'] + 1e-6)

# Customer Lifecycle Features
df_features['Account_Length_Months'] = df_features['Account Length'] / 30.0
df_features['Usage_Per_Day'] = df_features['Total_Minutes'] / (df_features['Account Length'] + 1)
df_features['Revenue_Per_Day'] = df_features['Total_Charges'] / (df_features['Account Length'] + 1)
df_features['Service_Calls_Per_Month'] = df_features['CustServ Calls'] / (df_features['Account_Length_Months'] + 1e-6)

print(f"✅ Created {df_features.shape[1] - df_clean.shape[1]} new features")
print(f"Total features now: {df_features.shape[1]}")

🔧 Engineering Advanced Features...
Re-confirmed df_features are duplicate-free: 60,445 records
✅ Created 19 new features
Total features now: 37


In [6]:
# Behavioral Indicators (Binary Flags)
print("🎯 Creating Behavioral Indicators...")

df_features['Is_Heavy_Day_User'] = (df_features['Day Mins'] > df_features['Day Mins'].quantile(0.75)).astype(int)
df_features['Is_Heavy_Eve_User'] = (df_features['Eve Mins'] > df_features['Eve Mins'].quantile(0.75)).astype(int)
df_features['Is_Heavy_Night_User'] = (df_features['Night Mins'] > df_features['Night Mins'].quantile(0.75)).astype(int)
df_features['Is_Intl_User'] = (df_features['Intl Mins'] > 0).astype(int)
df_features['Is_High_Service_User'] = (df_features['CustServ Calls'] > 2).astype(int)
df_features['Has_Voicemail'] = (df_features['VMail Message'] > 0).astype(int)
df_features['Is_High_Value_Customer'] = (df_features['Total_Charges'] > df_features['Total_Charges'].quantile(0.8)).astype(int)

# Risk Indicators
df_features['Churn_Risk_Score'] = (
    df_features['CustServ Calls'] * 0.4 +
    (df_features['Total_Minutes'] < df_features['Total_Minutes'].quantile(0.25)) * 0.3 +
    (df_features['Account Length'] < 100) * 0.3
)

# Customer value segments
df_features['Customer_Value_Score'] = (
    df_features['Total_Charges'] * 0.4 +
    df_features['Account Length'] * 0.3 +
    df_features['Total_Minutes'] * 0.3
)

# Engagement metrics
df_features['Engagement_Score'] = (
    df_features['Has_Voicemail'] * 0.2 +
    df_features['Is_Intl_User'] * 0.3 +
    (df_features['CustServ Calls'] == 0) * 0.5
)

print(f"✅ Total features after engineering: {df_features.shape[1]}")

🎯 Creating Behavioral Indicators...
✅ Total features after engineering: 47


In [7]:
# Feature Summary
print("📊 Feature Engineering Summary")
print("="*50)

feature_categories = {
    'Original Features': 17,
    'Usage Aggregation': 3,
    'Usage Intensity': 4, 
    'Usage Patterns': 4,
    'Revenue Features': 4,
    'Lifecycle Features': 4,
    'Behavioral Flags': 7,
    'Risk Indicators': 1,
    'Value Scores': 2,
    'Engagement Metrics': 1
}

for category, count in feature_categories.items():
    print(f"{category}: {count} features")

print(f"\nTotal Engineered Features: {sum(feature_categories.values())}")
print(f"Actual DataFrame Shape: {df_features.shape}")

📊 Feature Engineering Summary
Original Features: 17 features
Usage Aggregation: 3 features
Usage Intensity: 4 features
Usage Patterns: 4 features
Revenue Features: 4 features
Lifecycle Features: 4 features
Behavioral Flags: 7 features
Risk Indicators: 1 features
Value Scores: 2 features
Engagement Metrics: 1 features

Total Engineered Features: 47
Actual DataFrame Shape: (60445, 47)


In [8]:
# Save processed data
print("💾 Saving processed data...")

# Create processed directory if it doesn't exist
import os
os.makedirs('../data/processed', exist_ok=True)

# Save the processed dataset
df_features.to_csv('../data/processed/telecom_processed.csv', index=False)
print(f"✅ Saved processed dataset: {df_features.shape}")

# Save feature names for ML pipeline
feature_names = list(df_features.columns)
with open('../data/processed/feature_names.txt', 'w') as f:
    for feature in feature_names:
        f.write(f"{feature}\n")

print(f"✅ Saved {len(feature_names)} feature names")
print("\n🎉 Data preprocessing completed successfully!")

💾 Saving processed data...


✅ Saved processed dataset: (60445, 47)
✅ Saved 47 feature names

🎉 Data preprocessing completed successfully!
